## Importing Modules

In [35]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from groq import Groq
import os
from typing import TypedDict,Annotated,Optional,Literal
from pydantic import BaseModel, EmailStr, Field


load_dotenv()

True

In [2]:
model = ChatGroq(
        groq_api_key=os.getenv("GROQ_API_KEY"),
        model_name="Gemma2-9b-It",
        temperature=0.5 ## this is creative parameter
    )

## Typed Dict

In [13]:
## schema


class Review(TypedDict):
    
    summary : Annotated[str,"A brief summary of Review"] ## type and prompt
    sentiment : Annotated[str,"A brief summary of Review eigther negative , positive or neutral"]


structered_model = model.with_structured_output(Review) ## we are invoking the above class to get struteced output

In [14]:
result = structered_model.invoke(
    '''Hardware is good but the software fooled us. There are too many installed apps which i cant remove

    '''

)
print(result) ## you can see that output is dictnory format

{'sentiment': 'negative', 'summary': 'Hardware is good but the software fooled us. There are too many installed apps which i cant remove'}


In [15]:
## schema


class Review(TypedDict):

    key_themes : Annotated[list[str],"Write down all the key points discussed in the review"]
    summary : Annotated[str,"A brief summary of Review"] ## type and prompt
    sentiment : Annotated[str,"A brief summary of Review eigther negative , positive or neutral"]


structered_model = model.with_structured_output(Review) ## we are invoking the above class to get struteced output

In [16]:
review = '''
The Samsung Galaxy S24 Ultra is a premium flagship with cutting-edge hardware and AI-enhanced features. With its 200MP camera and Snapdragon 8 Gen 3 chip, it’s aimed at power users and mobile photographers.

Pros

Stunning 6.8″ AMOLED display with 120 Hz refresh rate

200MP camera captures extraordinary detail

Long battery life with 5,000 mAh capacity

Galaxy AI tools for image editing, live translation, and more

7 years of software updates

Cons

Expensive compared to rivals

Bulky and heavy to hold

AI features sometimes feel undercooked

Slow 45W charging for the price



'''

In [18]:
result = structered_model.invoke(
    review

)
print(result) ## you can see that output is dictnory format

{'key_themes': ['camera', 'display', 'battery life', 'software updates', 'price'], 'sentiment': 'neutral', 'summary': 'A review of the Samsung Galaxy S24 Ultra highlighting its pros and cons.'}


## Pydantic

In [30]:
class Student(BaseModel): ## we inherit the base model

    name: str 
    age: Optional[int] = None  ## its optional if age is not given it returns none
    email: EmailStr  ## inbuilt it validates the email
    cgpa: float = Field(gt=0, lt=10, default=5, description='A decimal value representing the cgpa of the student')
    ## we add constrains using field function like here gpa much be between 0and 10



In [ ]:
new_student = {'name':'chinu','age':'32', 'email':'abc'} ## throws error as its not in email format

student = Student(**new_student)

In [31]:
new_student = {'name':'chinu','age':'32', 'email':'abc@gmail.com'}

student = Student(**new_student)
student

Student(name='chinu', age=32, email='abc@gmail.com', cgpa=5)

In [37]:
new_student = {'name':'chinu', 'email':'abc@gmail.com'}

student = Student(**new_student)
student

Student(name='chinu', age=None, email='abc@gmail.com', cgpa=5)

In [33]:
new_student = {'name':'chinu','age':32, 'email':'abc@gmail.com',"cgpa":9}

student = Student(**new_student)

student_dict = dict(student)

print(student_dict)

{'name': 'chinu', 'age': 32, 'email': 'abc@gmail.com', 'cgpa': 9.0}


In [39]:
class Review(BaseModel):

    key_themes: list[str] = Field(description="Write down all the key themes discussed in the review in a list")
    summary: str = Field(description="A brief summary of the review")
    sentiment: Literal["pos", "neg"] = Field(description="Return sentiment of the review either negative, positive or neutral")
    pros: Optional[list[str]] = Field(default=None, description="Write down all the pros inside a list")
    cons: Optional[list[str]] = Field(default=None, description="Write down all the cons inside a list")
    name: Optional[str] = Field(default=None, description="Write the name of the reviewer")
    

structured_model = model.with_structured_output(Review)

result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Nitish Singh
""")

print(result)
# result

key_themes=['Performance', 'Camera', 'Battery Life', 'S-Pen', 'Software', 'Price'] summary='The Samsung Galaxy S24 Ultra is a powerful phone with a stunning camera and long battery life, but its size and price may be drawbacks for some users.' sentiment='pos' pros=['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'] cons=['Weight and size make it uncomfortable for one-handed use', 'Samsung’s One UI still comes with bloatware', 'The $1,300 price tag is a hard pill to swallow'] name='Nitish Singh'


## JSON

In [41]:
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": [
        "pos",
        "neg"
      ],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": [
        "array",
        "null"
      ],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": [
        "array",
        "null"
      ],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": [
        "string",
        "null"
      ],
      "description": "Write the name of the reviewer"
    }
  },
  "required": [
    "key_themes",
    "summary",
    "sentiment"
  ]
}

structured_model = model.with_structured_output(json_schema)

result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful
                                 
Review by Nitish Singh
""")

print(result)

{'cons': ['Weight and size make it uncomfortable for one-handed use', "Bloatware from Samsung's One UI", 'High price tag of $1,300'], 'key_themes': ['Performance', 'Camera', 'Battery Life', 'S-Pen', 'Price'], 'name': 'Nitish Singh', 'pros': ['Insanely powerful processor (great for gaming and productivity)', 'Stunning 200MP camera with incredible zoom capabilities', 'Long battery life with fast charging', 'S-Pen support is unique and useful'], 'sentiment': 'pos', 'summary': 'The Samsung Galaxy S24 Ultra is a powerful phone with a great camera and long battery life. However, it is heavy and expensive.'}
